In [8]:
# in order to work with arff files we have to install 'liac-arff' - if first usage: uncomment code cell below and run
# pip install liac-arff

Note: you may need to restart the kernel to use updated packages.


In [1]:
import arff
import numpy as np
import pandas as pd

In [2]:
data = arff.load(open('data/diabetes_dataset.arff', 'r'))

In [3]:
df_header = list(pd.DataFrame(data['attributes'])[0])
df_header

['Age',
 'Gender',
 'Family_Diabetes',
 'highBP',
 'PhysicallyActive',
 'BMI',
 'Smoking',
 'Alcohol',
 'Sleep',
 'SoundSleep',
 'RegularMedicine',
 'JunkFood',
 'Stress',
 'BPLevel',
 'Pregancies',
 'Pdiabetes',
 'UriationFreq',
 'Diabetic']

In [4]:
df = pd.DataFrame(data['data'], columns=df_header)
df

,Age,Gender,Family_Diabetes,highBP,PhysicallyActive,BMI,Smoking,Alcohol,Sleep,SoundSleep,RegularMedicine,JunkFood,Stress,BPLevel,Pregancies,Pdiabetes,UriationFreq,Diabetic
0,50-59,Male,no,yes,one hr or more,39.0,no,no,8,6,no,occasionally,sometimes,high,0.0,0,not much,no
1,50-59,Male,no,yes,less than half an hr,28.0,no,no,8,6,yes,very often,sometimes,normal,0.0,0,not much,no
2,40-49,Male,no,no,one hr or more,24.0,no,no,6,6,no,occasionally,sometimes,normal,0.0,0,not much,no
3,50-59,Male,no,no,one hr or more,23.0,no,no,8,6,no,occasionally,sometimes,normal,0.0,0,not much,no
4,40-49,Male,no,no,less than half an hr,27.0,no,no,8,8,no,occasionally,sometimes,normal,0.0,0,not much,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,less than 40,Male,yes,no,more than half an hr,25.0,no,no,8,6,no,often,sometimes,normal,0.0,0,not much,yes
948,60 or older,Male,yes,yes,more than half an hr,27.0,no,no,6,5,yes,occasionally,sometimes,high,0.0,0,quite often,yes
949,60 or older,Male,no,yes,none,23.0,no,no,6,5,yes,occasionally,sometimes,high,0.0,0,not much,no
950,60 or older,Male,no,yes,less than half an hr,27.0,no,yes,6,5,yes,occasionally,very often,high,0.0,0,not much,no


In [5]:
df['Diabetic'].unique()

array(['no', 'yes', ' no', None], dtype=object)